In [8]:
%pip install lobsang openai python-dotenv jsonschema

Note: you may need to restart the kernel to use updated packages.


# Answers

Answers are an important concept for Lobsang. They are used to guide the LLM to produce answers in a specific format. For example, you can use an answer to ask the LLM to produce a JSON object. To do so, an answer generally does two things:

1. It directs the LLM to produce a specific output format. For example, the `JSONAnswer` directs the LLM to produce a JSON object. It does so by adding a [JSON schema](https://json-schema.org/) as well as an example to the user message.
2. It tried to parse the LLM output corresponding to the modified user message from step 1 to the desired format. For example, the `JSONAnswer` tries to parse the LLM output to a JSON object using the provided schema. It will also validate the output against the schema.

Let's take a look at the first step. The `JSONAnswer` takes a schema and an example as arguments and embeds them into the user message:
 

In [9]:
from lobsang.answers import JSONAnswer

# For this example, we define a JSON schema for a hero
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "The name of the hero"},
        "age": {"type": "integer", "minimum": 0, "description": "The age of the hero"},
        "powers": {
            "type": "array",
            "items": {"type": "string"},
            "description": "The powers of the hero"
        }
    }
}

# and an example according to the schema
example = {
    "name": "Spiderman",
    "age": 18,
    "powers": ["spider sense", "web slinging", "superhuman strength"]
}

# Create JSON answer
hero_answer = JSONAnswer(schema=schema, example=example)

# With the answer set up, we can now create the directive (i.e. the modified user message)
original = "Create a marvel hero"
#  ☝️ becomes 👇
directive = hero_answer.directive(original)

print(directive)

Create a marvel hero

Create a JSON object with the following schema:
```json
{'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the hero'}, 'age': {'type': 'integer', 'minimum': 0, 'description': 'The age of the hero'}, 'powers': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The powers of the hero'}}}
```

Example:
```json
{'name': 'Spiderman', 'age': 18, 'powers': ['spider sense', 'web slinging', 'superhuman strength']}
```


☝️ As you can see the instructions are added to the user message. Note that this will not update the user message in the chat history. However, it is added as 'directive' to the `info` object of the response.

👇 Alright, let's also take a look at how the `JSONAnswer` parses the LLM output.

In [10]:
# Some response from the LLM
response = """
Sure, i can create a marvel hero for you. Here is an example:
```json
{
    "name": "Spiderman",
    "age": 18,
    "powers": ["spider sense", "web slinging", "superhuman strength"]
}
```
"""

# Parse response
parsed_response = hero_answer.parse(response)

print(parsed_response)

{'name': 'Spiderman', 'age': 18, 'powers': ['spider sense', 'web slinging', 'superhuman strength']}


☝️ As you can see, the `JSONAnswer` parses the LLM output to a dict. It will also validate the output against the schema. If the LLM output is not valid JSON, the answer will raise an error. 

**Note:** This can be cumbersome if you want to process multiple messages. In [3_subroutines](3_subroutines.ipynb) we will see how to ignore/resolve errors on the fly (e.g. in a message chain).

👇 Great! Now let's give it a try! 🚀 Make sure to update the `.env` file with your own OpenAI API key, see [1_basics](1_basics.ipynb) for more details.

In [11]:
import os

from dotenv import load_dotenv

from lobsang import Chat, OpenAI, UserMessage

load_dotenv()

# Load OpenAI API key from .env file (please update .env file with your own API key)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Please set OPENAI_API_KEY in .env file"

print("All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY={OPENAI_API_KEY[:15]}...", end="\n\n")

# Create a chat instance with OpenAI LLM
llm = OpenAI(api_key=OPENAI_API_KEY)
chat=Chat(llm=llm)

# Create a conversation
messages = [
    UserMessage("Create a marvel hero."),
    hero_answer
]

# Let's chat! 🚀
chat(messages)
print(chat)

All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY={OPENAI_API_KEY[:15]}...
USER: Create a marvel hero.
ASSISTANT: Here is an example of a Marvel hero JSON object that follows the given schema:

```json
{
  "name": "Iron Man",
  "age": 40,
  "powers": [
    "Powered Armor",
    "Genius-level Intellect",
    "Flight",
    "Energy Blasts",
    "Superhuman Durability"
  ]
}
```

This JSON object represents a hero named "Iron Man" who is 40 years old and possesses powers like Powered Armor, Genius-level Intellect, Flight, Energy Blasts, and Superhuman Durability.


️️️☝️ Please note that this may throw an error if the LLM's output does not include a JSON block with a valid JSON object. This can happen if the LLM does not produce the desired output. For example, if the LLM does not produce a syntactically valid JSON object.

👇 Let's take a closer look at the response from the LLM which we can access via the chat history.

In [17]:
response = chat.history[-1]

# print("=== Text ===")
# print(response.text)
print("=== Data ===")
print(response.data)
print("=== Info ===")
print(response.info)

=== Data ===
{'name': 'Iron Man', 'age': 40, 'powers': ['Powered Armor', 'Genius-level Intellect', 'Flight', 'Energy Blasts', 'Superhuman Durability']}
=== Info ===
{'directive': "Create a marvel hero.\n\nCreate a JSON object with the following schema:\n```json\n{'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the hero'}, 'age': {'type': 'integer', 'minimum': 0, 'description': 'The age of the hero'}, 'powers': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The powers of the hero'}}}\n```\n\nExample:\n```json\n{'name': 'Spiderman', 'age': 18, 'powers': ['spider sense', 'web slinging', 'superhuman strength']}\n```"}


️️️☝️ Here the parsed JSON object is stored in the `data` object. The `info` object contains the modified user message (i.e. the directive) which was used to produce the response.

## Conclusion

Congratulations! 🎉 You just created your first chat with a `JSONAnswer`! 🚀 

In this notebook, we learned how to embed a user message with a `JSONAnswer` and how to parse the LLM's response. We also learned how to create a chat with a `JSONAnswer` and how to access the original message in the chat history. 

**Note:** In this example we used OpenAI's LLM. However, the `JSONAnswer` can be used with any LLM. You can also adapt the instructions of a directive via the `instructions` argument. Make it sure it contains  `{message}`,  `{example}` and `{schema}` (see DEFAULT_INSTRUCTIONS in [answers/json.py](../lobsang/answers/json.py)).

If you have any questions don't hesitate to reach out to us on [Discord](https://discord.gg/wMHVAaqh).
If you've found a bug, a spelling mistake or suggestions on what could be improved, please open an issue or a pull request on [GitHub](https://github.com/cereisen/lobsang).

See you in the next part! 👋 We hope we got you hooked 🎣 